In [2]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode="nearest")

Using TensorFlow backend.


In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [24]:
# dimensions of our images.
img_width, img_height = 64, 64

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 7500
nb_validation_samples = 1000
epochs = 50
batch_size = 16

In [5]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [25]:
model = Sequential()
model.add(Conv2D(128, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(81))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
activation_26 (Activation)   (None, 62, 62, 128)       0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
activation_27 (Activation)   (None, 29, 29, 256)       0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 14, 14, 256)      

In [16]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [17]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 162000 images belonging to 81 classes.
Found 32400 images belonging to 81 classes.


In [26]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    workers=4)

model.save_weights('first_try.h5')

Epoch 1/50
468/468 [==============================] - 131s 279ms/step - loss: 4.3994 - accuracy: 0.0100 - val_loss: 4.3911 - val_accuracy: 0.0141
Epoch 2/50
468/468 [==============================] - 130s 278ms/step - loss: 4.3956 - accuracy: 0.0118 - val_loss: 4.3885 - val_accuracy: 0.0161
Epoch 3/50
468/468 [==============================] - 129s 276ms/step - loss: 4.3957 - accuracy: 0.0122 - val_loss: 4.3916 - val_accuracy: 0.0171
Epoch 4/50
468/468 [==============================] - 129s 275ms/step - loss: 4.3959 - accuracy: 0.0146 - val_loss: 4.3985 - val_accuracy: 0.0171
Epoch 5/50
468/468 [==============================] - 128s 274ms/step - loss: 4.3951 - accuracy: 0.0134 - val_loss: 4.4063 - val_accuracy: 0.0161
Epoch 6/50
468/468 [==============================] - 124s 265ms/step - loss: 4.3948 - accuracy: 0.0111 - val_loss: 4.3827 - val_accuracy: 0.0131
Epoch 7/50
468/468 [==============================] - 125s 267ms/step - loss: 4.3954 - accuracy: 0.0126 - val_loss: 4.3905 -

KeyboardInterrupt: 

In [28]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 64, 64

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 7500*81
nb_validation_samples = 1000*81
epochs = 50
batch_size = 16



In [29]:

def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)




In [53]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array(
        [i] * (nb_train_samples / 81) for i in range(81))

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array(
        [i] * (nb_validation_samples / 81) for i in range(81))
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(81, activation='softmax'))

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])
    print(validation_labels)
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)



In [54]:

#save_bottlebeck_features()
train_top_model()

<generator object train_top_model.<locals>.<genexpr> at 0x0000024C61BBB4C8>


ValueError: Error when checking target: expected dense_30 to have 2 dimensions, but got array with shape ()